# Visualizing the Rolling Buffer Cache

We keep track of a write pointer that tells us where we added the last token in the rolling buffer cache. We add a new token and move the pointer forward.

Let’s add the sentence: 
        
        The cat is on a chair

Imagine we want to “unroll” the cache because we want to calculate the attention of the incoming token. It’s very easy! We just need to use the write pointer to understand how to order the items: **We first take all the items AFTER the write pointer, and then all the items from the 0th index up to the position of the write pointer (inclusive).**

In [1]:
def unrotate(cache: list, cache_size: int, seqlen: int, position: int) -> list:
    if seqlen < cache_size:  # since the cache is not full yet, ignore unfilled items
        return cache[: position + 1]

    else:
        # Take all the items AFTER the write pointer, and then all the items from the 0th index up to the position of the write pointer (inclusive)
        return cache[position + 1 :] + cache[: position + 1]

In [2]:
sequence = ["the", "cat", "is", "on", "a", "chair"]

cache = []  # start with an empty cache
cache_size = 4  # same as sliding window size

for i, token in enumerate(sequence):
    seqlen = i + 1  # how long is the sequence now
    position = i % cache_size  # position of write pointer (indexed 0)
    print("write pointer position:", position)  # print to see how the pointer moves

    # Continue appending to cache until it is filled
    if len(cache) < cache_size:
        cache.append(token)
    # Once cache is filled, replace oldest token with latest
    else:
        cache.pop(position)  # pop oldest token
        cache.insert(position, token)  # insert latest token

    print("Current cache (before unrolling):", cache)
    res = unrotate(cache, cache_size, seqlen, position)
    print("New cache (after unrolling):", res)
    print("===")

write pointer position: 0
Current cache (before unrolling): ['the']
New cache (after unrolling): ['the']
===
write pointer position: 1
Current cache (before unrolling): ['the', 'cat']
New cache (after unrolling): ['the', 'cat']
===
write pointer position: 2
Current cache (before unrolling): ['the', 'cat', 'is']
New cache (after unrolling): ['the', 'cat', 'is']
===
write pointer position: 3
Current cache (before unrolling): ['the', 'cat', 'is', 'on']
New cache (after unrolling): ['the', 'cat', 'is', 'on']
===
write pointer position: 0
Current cache (before unrolling): ['a', 'cat', 'is', 'on']
New cache (after unrolling): ['cat', 'is', 'on', 'a']
===
write pointer position: 1
Current cache (before unrolling): ['a', 'chair', 'is', 'on']
New cache (after unrolling): ['is', 'on', 'a', 'chair']
===
